## load txt into python in form of list

In [1]:
import pandas as pd
import logging
import numpy as np
import sys
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random
import math
import time
import numpy
from scipy.optimize import minimize
import math
import scipy
import sklearn
from os import listdir
from collections import Counter

In [2]:
dir_name = "F:/ML/hw3-sentiment/pos/"

In [3]:
def read_all_txt_in_a_directory(dir_name):
    txts = []
    for i in range(0, len(listdir(dir_name))):
        txts.append(open(dir_name + listdir(dir_name)[i], "r").read())
    return txts

In [4]:
pos_raws = read_all_txt_in_a_directory(dir_name)

In [5]:
poss= []
for pos in pos_raws:
    poss.append([pos,1])

In [6]:
dir_name = "F:/ML/hw3-sentiment/neg/"

In [7]:
neg_raws = read_all_txt_in_a_directory(dir_name)

In [8]:
negs= []
for neg in neg_raws:
    negs.append([neg,-1])

In [9]:
combined = []
combined = negs + poss
np.random.shuffle(combined)

In [10]:
train = []
test = []

In [11]:
for i in range(0,len(combined)):
    if len(train) == 1500:
        test.append(combined[i:i+1][0])
    elif len(test) == 500:
        train.append(combined[i:i+1][0])
    else:
        if random.random() < 0.75:
            train.append(combined[i:i+1][0])
        else:
            test.append(combined[i:i+1][0])

In [12]:
len(train)

1500

In [13]:
len(test)

500

In [14]:
counter_train = []
for i in range(0, len(train)):
    view = train[i][0].replace("\\", "")
    view = train[i][0].split()
    counter_train.append(Counter())
    for word in view:
        if len(word) == 1:
            continue
        else:
            current = word.lower()
            if current[-1] == "\'":
                current = current[0:len(current)-1]
            if current[0] == "\n":
                counter_train[i][current[-len(current)+1:]] += 1
            elif current[0] >= "a" and current[0] <= "z":
                counter_train[i][current] += 1
        

In [172]:
"!"

'!'

In [15]:
counter_test = []
for i in range(0, len(test)):
    view = test[i][0].replace("\\", "")
    view = test[i][0].split()
    counter_test.append(Counter())
    for word in view:
        if len(word) == 1 and word != "!" and word != "?":
            continue
        else:
            current = word.lower()
            if current[-1] == "\'":
                current = current[0:len(current)-1]
            if current[0] == "\n":
                counter_test[i][current[-len(current)+1:]] += 1
            elif current[0] >= "a" and current[0] <= "z":
                counter_test[i][current] += 1        

In [16]:
def Pegasos(counter_train,train,num_iter=1000,lambda_reg=0.1,tol=1e-4):
    w = Counter()
    W = Counter()
    losses = []
    s_t = 1
    for itr in range(0,num_iter):
        w_lag = sum([abs(number) for number in w.values()])
        etha = 1/((itr+1)*lambda_reg)
        s_t = (1 - etha * lambda_reg)*s_t
        if s_t == 0:
            s_t = 1
            W = Counter()
            continue
        for i in range(0,len(train)):
            loss = 0
            for key in counter_train[i].keys():
                loss = loss + w[key] * counter_train[i][key]
            loss = loss * train[i][1]
            losses.append(loss)
            if loss < 1:
                root = math.sqrt(sum([(number*number) for number in counter_train[i].values()]))
                for key in counter_train[i].keys():
                    W[key] = W[key] + (s_t ** -1) * etha * train[i][1] * counter_train[i][key] / root
            for key in W.keys():
                w[key] = s_t * W[key]
            w_absum = sum([abs(number) for number in w.values()])
            if len(w.items()) != 0 and w_lag != 0 and abs((w_absum/w_lag)-1)/(len(w.items())) < tol:
                print("%s's epoch" % itr)
                return w,losses
    return w,losses

The following implementation is without the s_t trick

In [142]:
start_time = time.time()
wa,lo = Pegasos(counter_train,train,num_iter=20,lambda_reg=0.1,tol=1e-8)
print("--- %s seconds ---" % (time.time() - start_time))

2's epoch
--- 146.256999969 seconds ---


The following implementation is with the s_t trick

In [257]:
start_time = time.time()
wa,lo = Pegasos(counter_train,train,num_iter=1000,lambda_reg=0.1,tol=1e-10)
print("--- %s seconds ---" % (time.time() - start_time))

51's epoch
--- 8757.43400002 seconds ---


In [46]:
def Pecasos_prediction_error(w,c,test):
    objs = []
    for i in range(0,len(c)):
        obj = 0
        for key in c[i].keys():
            obj = obj + w[key] * c[i][key]
        objs.append(obj)
    right = 0
    for i in range(0,len(objs)):
        if objs[i] >1 and test[i][1] ==1:
            right += 1
        elif objs[i] <-1 and test[i][1] ==-1:
            right += 1
    return (1 - float(right)/len(objs)),objs

In [264]:
Pecasos_prediction_error(wa,counter_test)

0.31799999999999995

In [19]:
import csv
w = csv.writer(open("output.csv", "w"))
for key, val in wa.items():
    w.writerow([key, val])

In [20]:
import csv
w = {}
for key, val in csv.reader(open("output.csv")):
    w[key] = val

In [21]:
w

{'circuitry': '-0.06439796231327606',
 'spiders': '0.046584749531245626',
 "frasier's": '-0.059054889272526',
 'woody': '0.70308324270159',
 'comically': '-0.04194059807020658',
 'co-venture': '0.04882989460908731',
 'hennings': '-0.09102576894656005',
 'scold': '0.045369323667338264',
 'jell-o': '-0.038058595210789574',
 'originality': '-0.1853134935060269',
 'rickman': '-0.007667383815010921',
 "franklin's": '-0.05365189833163754',
 'capoeira': '0.03683297467398867',
 'rawhide': '0.103114704148291',
 'bringing': '-0.04774240681779602',
 'liaisons': '-0.03792645743354371',
 'wooden': '0.02488197601559809',
 'wednesday': '0.0517433683804222',
 'indie-underground': '-0.03219898115663803',
 'woods': '-0.3357041873772297',
 'crotch': '0.03580922808152935',
 'stereotypical': '-0.24953469941215944',
 'busybody': '0.023351168761959164',
 'sooty': '0.05063891604225661',
 "tom's": '0.34808280552489224',
 'snuggles': '-0.09699599366322091',
 'hanging': '-0.2931580871220229',
 'miniatures': '-0.

Using the bag-of-words feature representation described above, search for the regularization
parameter that gives the minimal percent error on your test set. A good search strategy is to
start with a set of lambdas spanning a broad range of orders of magnitude. Then, continue to
zoom in until you’re convinced that additional search will not significantly improve your test
performance. Once you have a sense of the general range of regularization parameters that
give good results, you do not have to search over orders of magnitude every time you change
something (such as adding new feature)

In [22]:
def find_l2_minimizer_lambda(counter_train,train,counter_test, lambda_start_log = -7.0,lambda_end_log = 3.0,num_iter=10):
    for i in range(0,num_iter):
        loss={"valid":[]}
        r = (lambda_end_log-lambda_start_log)/10
        lambda_range = []
        plots = []
        for j in range(0,10):
            lambda_range.append(math.exp(math.log(10)*(lambda_start_log+r*j)))
        for j in lambda_range:
            a,b=Pegasos(counter_train,train,num_iter=20,lambda_reg=j,tol=1e-10)
            loss["valid"].append(Pecasos_prediction_error(a,counter_test))
            
        minl = loss["valid"].index(min(loss["valid"]))

        if minl == 0:
            lambda_end_log = lambda_start_log+r
        elif minl == len(lambda_range)-1:
            lambda_start_log = lambda_end_log-r
        else:
            lambda_start_log = lambda_start_log + (minl-1)*r
            lambda_end_log = lambda_start_log + 2 * r

    print i
    
    return lambda_range,loss["valid"]


In [ ]:
a,b=find_l2_minimizer_lambda(counter_train,train,counter_test, lambda_start_log = -7.0,lambda_end_log = 3.0,num_iter=1)

In [ ]:
import os
os.system("printf '\a'") # or '\7'

In [ ]:
a

In [ ]:
b

Recall that the “score” is the value of the prediction f(x) = wT x. We like to think
that the magnitude of the score represents the confidence of the prediction. This is something
we can directly verify or refute. Break the predictions into groups based on the score (you
can play with the size of the groups to get a result you think is informative).

In [82]:
a,b = Pecasos_prediction_error(wa,counter_test,test)

In [48]:
import os
os.system("printf '\a'") # or '\7'

1

In [78]:
def Pecasos_prediction_error_by_magnitude(w,c,test,interval=50):
    objs = []
    for i in range(0,len(c)):
        obj = 0
        for key in c[i].keys():
            obj = obj + w[key] * c[i][key]
        objs.append(obj)
    
    error = {}
    for j in range(1,int(max(objs)/interval)/2):    
        right = 0
        count = 0
        for i in range(0,len(objs)):
            if abs(objs[i]/interval) < j and abs(objs[i]/interval) > j-1 :
                count += 1
                if objs[i] >1 and test[i][1] ==1:
                    right += 1
                elif objs[i] <-1 and test[i][1] ==-1:
                    right += 1
        print right,count
        error[j] = 1- (float(right) / count)
    return error


In [79]:
e = Pecasos_prediction_error_by_magnitude(wa,counter_test,test,interval=20)

49 106
57 93
54 73
48 63
43 52
32 33
18 21
17 19
8 8


In [80]:
e

{1: 0.5377358490566038,
 2: 0.3870967741935484,
 3: 0.26027397260273977,
 4: 0.23809523809523814,
 5: 0.17307692307692313,
 6: 0.030303030303030276,
 7: 0.1428571428571429,
 8: 0.10526315789473684,
 9: 0.0}

### 7 Error Analysis

In [85]:
for i in range(0,len(b)):
    print b[i],test[i][1]

23.3081460806 -1
-11.8077521123 1
70.7165297492 1
-6.98631127915 -1
81.3312840651 1
109.22765957 1
96.3702006584 1
11.2104385146 -1
-84.4227681066 -1
165.673948759 1
279.794668327 1
69.2929440359 1
66.0180784377 1
93.8803780449 1
53.9316091474 1
52.5161013957 -1
-87.1358846111 -1
29.4154172248 -1
225.516751738 1
128.222891434 1
161.519810137 1
103.116520081 1
74.4184706764 1
-39.5293062786 -1
-35.2764249358 -1
44.6734831519 -1
-3.52277123473 -1
50.5465216077 -1
-41.4917940274 -1
27.3376879632 -1
77.8857139492 1
58.9980083363 1
50.0375072241 1
-23.1683680121 -1
76.2482178892 1
281.618638773 1
65.7142077123 1
-30.0075941148 1
43.3868231468 -1
-28.1423925265 -1
-18.9078622487 -1
155.375474475 1
111.107530342 1
44.3202285278 -1
-4.18502241846 -1
66.0075275097 1
16.2389270271 -1
-42.7926071398 -1
76.2920581751 1
-28.1088774123 -1
8.04179494273 -1
-147.194655718 -1
-33.1722268281 -1
12.5470322696 1
46.9389771822 -1
94.8016926703 1
-29.950995557 -1
-7.50154087654 1
100.888281217 1
4.015451337

In [86]:
test[0]

["starring shawnee smith ; donovan leitch ; ricky paull goldin ; kevin dillon & billy beck the blob is the remake of the 1960's classic ( a term that i use very loosely to define the original ) about a really mean glob of goop that takes out anything that gets in its way . \nnow the original version has the virtue of cheesy special effects which give it a kind of nostalgic campy feel . \nthe fact that steve mcqueen was the star of the film doesn't exactly hurt it either . \nfast forward to the late '80's . \nsteve mcqueen isn't in the remake ( this might have something to do with the fact that he had been pushing up daisies for years ) . \nnor is there any hollywood heavy hitters in the lead role . \nthat is unless you count matt dillon's brother kevin as a hollywood heavy hitter . \nthe other thing that works against this new blob is that special effects technology has improved dramatically since the original . \nthe only things you need now to have great special effects in a movie ar

In [143]:
objs0 = Counter()
keys = []
for i in range(0,len(test)):
    for key in counter_test[0].keys():
        obj =  wa[key] * counter_test[0][key]
        objs0[key]=obj

In [144]:
for j in range(0,15):
    value0 = sorted(objs0.values())[j]
    for key, value in objs0.iteritems():
        if value == value0:
            keys.append(key)
            print key
            break
    value0 = sorted(objs0.values())[-j]
    for key, value in objs0.iteritems():
        if value == value0:
            keys.append(key)
            print key
            break

this
this
that
of
have
the
any
is
are
you
movie
very
about
it
out
and
isn't
as
had
more
can
to
original
which
only
from
up
in
if
great


In [145]:
objs0 = Counter()
for i in range(0,len(test)):
    for key in counter_test[1].keys():
        obj =  wa[key] * counter_test[1][key]
        objs0[key]=obj
for j in range(0,15):
    value0 = sorted(objs0.values())[j]
    for key, value in objs0.iteritems():
        if value == value0:
            keys.append(key)
            print key
            break
    value0 = sorted(objs0.values())[-j]
    for key, value in objs0.iteritems():
        if value == value0:
            keys.append(key)
            print key
            break

bad
bad
this
of
that
and
out
the
be
is
then
you
into
to
her
as
all
film
have
best
at
from
there's
it
any
most
about
seen
know
for


In [146]:
keys = list(set(keys))

In [147]:
keys

['and',
 'all',
 'then',
 'is',
 'it',
 'as',
 'are',
 'have',
 'in',
 'seen',
 'any',
 'film',
 'out',
 'from',
 'her',
 'movie',
 'had',
 'to',
 'only',
 "there's",
 'best',
 'which',
 'you',
 'if',
 'into',
 'more',
 'be',
 'that',
 'very',
 'great',
 'most',
 'know',
 'about',
 'for',
 'this',
 'of',
 'up',
 'bad',
 'can',
 'the',
 'original',
 "isn't",
 'at']

In [148]:
for key, value in wa.items():
    if key in keys:
        wa[key] = 0

In [123]:
e1 = Pecasos_prediction_error_by_magnitude(wa,counter_test,test,interval=20)

93 172
95 132
75 92


In [149]:
a2,b2 = Pecasos_prediction_error(wa,counter_test,test)

In [125]:
a

0.28600000000000003

In [126]:
a1

0.28400000000000003

In [134]:
a2

0.32799999999999996

In [150]:
a2

0.23399999999999999

In [160]:
wa['worse'] = -114.2/10
wa['bore']  =-95.3/10
wa['devoid']  = -93.7/10
wa['ugly']  = -84.0/10
wa['wonderful']  = -76.4/10
wa['shallow']  = - 71.1/10
wa['obnoxious']  = -69.8/10
wa['rip-off']  = -69.8/10
wa['banal']  = -67.4/10
wa['shooting']  = -66.6/10
wa['treat']  = - 66.0/10
wa['wry']  =-58.2/10
wa['dullest']  = -57.8/10
wa['pretentious']  =-57.6/10
wa['loses']  = -57.2/10

In [161]:
a2,b2 = Pecasos_prediction_error(wa,counter_test,test)

In [162]:
a2

0.24